In [0]:
import pyspark,pyspark.sql
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('Movie_recomder').getOrCreate()

In [0]:
md = spark.read.csv("/FileStore/tables/movielens_ratings.csv",inferSchema ='True', header='True')

In [0]:
md.tail(20)

Out[10]: [Row(movieId=59, rating=1.0, userId=29),
 Row(movieId=61, rating=1.0, userId=29),
 Row(movieId=62, rating=1.0, userId=29),
 Row(movieId=67, rating=1.0, userId=29),
 Row(movieId=68, rating=3.0, userId=29),
 Row(movieId=69, rating=1.0, userId=29),
 Row(movieId=70, rating=1.0, userId=29),
 Row(movieId=74, rating=1.0, userId=29),
 Row(movieId=75, rating=1.0, userId=29),
 Row(movieId=79, rating=2.0, userId=29),
 Row(movieId=80, rating=1.0, userId=29),
 Row(movieId=81, rating=2.0, userId=29),
 Row(movieId=83, rating=1.0, userId=29),
 Row(movieId=85, rating=1.0, userId=29),
 Row(movieId=86, rating=1.0, userId=29),
 Row(movieId=90, rating=4.0, userId=29),
 Row(movieId=93, rating=1.0, userId=29),
 Row(movieId=94, rating=4.0, userId=29),
 Row(movieId=97, rating=1.0, userId=29),
 Row(movieId=99, rating=1.0, userId=29)]

In [0]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
md.describe().show()

+-------+------------------+------------------+------------------+
summary| movieId| rating| userId|
+-------+------------------+------------------+------------------+
 count| 1501| 1501| 1501|
 mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
 stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
 min| 0| 1.0| 0|
 max| 99| 5.0| 29|
+-------+------------------+------------------+------------------+

In [0]:
# train, test data with 80:20 split
training,test = md.randomSplit([0.8,0.2])

In [0]:
als = ALS(maxIter=5,regParam =0.01,userCol='userId',itemCol='movieId',ratingCol='rating')

In [0]:
models = als.fit(training)

In [0]:
predictions = models.transform(test)

In [0]:
#to improve the preduction ratings we need work with more data the dataset has only 1501 datapoints which is very less for recomender system. 
predictions.show()

+-------+------+------+-------------+
movieId|rating|userId| prediction|
+-------+------+------+-------------+
 31| 1.0| 0| 0.15395409|
 31| 1.0| 27|-0.0061139315|
 31| 1.0| 29| 2.9922147|
 85| 1.0| 5| 2.7693405|
 85| 3.0| 6| 0.50526476|
 85| 4.0| 7| 3.893636|
 85| 3.0| 21| 0.5745565|
 85| 1.0| 25| 0.38709992|
 85| 1.0| 26| -0.8838575|
 65| 1.0| 2| -0.54002863|
 65| 1.0| 22| -0.28116786|
 78| 1.0| 8| 0.7820178|
 78| 1.0| 20| 0.8684967|
 78| 1.0| 24| 1.3616104|
 34| 1.0| 28| 1.4690526|
 81| 1.0| 6| 0.754382|
 81| 2.0| 9| 0.09749746|
 81| 1.0| 21| -1.726337|
 81| 3.0| 26| 2.3641472|
 28| 1.0| 0| 1.4223518|
+-------+------+------+-------------+
only showing top 20 rows

In [0]:
evaluator = RegressionEvaluator(metricName='rmse',labelCol='rating',predictionCol='prediction')

In [0]:
#rsme= Root mean squre error
rsme = evaluator.evaluate(predictions)

In [0]:
print('RMSE: '+str(rsme))

RMSE: 1.8925590474814766

In [0]:
#for a sigle user how to predict the new movie he like or not:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [0]:
single_user.show()

+-------+------+
movieId|userId|
+-------+------+
 25| 11|
 27| 11|
 36| 11|
 43| 11|
 45| 11|
 51| 11|
 66| 11|
 77| 11|
 89| 11|
+-------+------+

In [0]:
recommendations = models.transform(single_user)

In [0]:
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+-----------+
movieId|userId| prediction|
+-------+------+-----------+
 36| 11| 2.9405916|
 51| 11| 2.113773|
 45| 11| 0.27867603|
 27| 11| 0.22221586|
 89| 11| -0.2453264|
 43| 11|-0.69550693|
 66| 11| -0.8022375|
 25| 11| -2.5795686|
 77| 11| -3.082553|
+-------+------+-----------+

In [0]:
#in above we can say user 11 like movieID 36 as the prediction is +ve score i.e +2.94 and do not like movie id 77 with prediction -ve score i.e -3.08 prediction for liking(+ve) or not linking(-ve) according to previous behaviour
# all the above compution based on the data given it might go wrong for verious parametes